In [1]:
!pip install transformers
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import transformers
import torch
from torch.utils.data import Dataset, DataLoader

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 37.1 MB/s eta 0:00:00


In [2]:
df = pd.read_csv("/content/drive/MyDrive/pro/8701/combined_14510_xlnet.csv")
title_only = df[(df['title_polyglot_detect'] == 'en') & (df['title_lang_detect'] == 'en') & (df['title_langid_detect'] == 'en') & (df['title_xl_detect'] == 'en')][['question_title', 'class_index']]

In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import transformers
from transformers import RobertaModel, RobertaTokenizer

# specify GPU
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [4]:
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained('roberta-base')
label2id = {
    1: 0,
    5: 1,
    6: 2,
    10: 3
}

id2label = {
    0: 1,
    1: 5,
    2: 6,
    3: 10
}
class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels = [label2id[label] for label in df['class_index']]
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df['question_title']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [5]:
np.random.seed(112)
df_train, df_val, df_test = np.split(title_only.sample(frac=1, random_state=42), 
                                     [int(.8*len(title_only)), int(.9*len(title_only))])

print(len(df_train),len(df_val), len(df_test)) #dataframes

13100 1637 1638


In [13]:
from torch import nn

class RoBertaClassifier(nn.Module):

    def __init__(self): #, dropout=0.5

        super(RoBertaClassifier, self).__init__()

        self.roberta = AutoModelForSequenceClassification.from_pretrained('roberta-base', num_labels = 4, label2id=label2id, id2label=id2label)
        # self.dropout = nn.Dropout(dropout)
        # self.linear = nn.Linear(768, 4)
        # self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        pooled_output = self.roberta(input_ids= input_id, attention_mask=mask,return_dict=False)
    #     dropout_output = self.dropout(pooled_output)
    #     linear_output = self.linear(dropout_output)
    #     final_layer = self.relu(linear_output)
        # print(pooled_output)
        # print(pooled_output.shape)
        # print(pooled_output[0])

        return pooled_output

    def save_model(self, path, tokenizer): #'/content/drive/MyDrive/pro/8701/roberta_model_content/'
        self.roberta.save_pretrained(path)
        tokenizer.save_pretrained(path)

In [14]:
from torch.optim import Adam
from tqdm import tqdm

def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=8, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=8)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    train_acc = []
    valid_acc = []

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)
                
                batch_loss = criterion(output[0], train_label.long())
                total_loss_train += batch_loss.item()
                
                acc = (output[0].argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output[0], val_label.long())
                    total_loss_val += batch_loss.item()
                    
                    acc = (output[0].argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}')
            
            train_acc.append(total_acc_train / len(train_data))
            valid_acc.append(total_acc_val / len(val_data))
            
            path = '/content/drive/MyDrive/pro/8701/roberta_model/saved_weights_' + str(epoch_num)
            print(path)
            # torch.save(model.state_dict(), path)
            model.save_model(path = path, tokenizer=tokenizer)

            
    return train_acc, valid_acc
                  
EPOCHS = 3
model = RoBertaClassifier()
LR = 1e-5

train(model, df_train, df_val, LR, EPOCHS)
# train(model, df_train.iloc[:10], df_val.iloc[:8], LR, EPOCHS)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'roberta.pooler.dense.weight', 'lm_head.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.

Epochs: 1 | Train Loss:  0.057                 | Train Accuracy:  0.840                 | Val Loss:  0.044                 | Val Accuracy:  0.883
/content/drive/MyDrive/pro/8701/roberta_model/saved_weights_0


100%|██████████| 1638/1638 [18:10<00:00,  1.50it/s]


Epochs: 2 | Train Loss:  0.030                 | Train Accuracy:  0.920                 | Val Loss:  0.047                 | Val Accuracy:  0.871
/content/drive/MyDrive/pro/8701/roberta_model/saved_weights_1


100%|██████████| 1638/1638 [18:11<00:00,  1.50it/s]


Epochs: 3 | Train Loss:  0.014                 | Train Accuracy:  0.966                 | Val Loss:  0.055                 | Val Accuracy:  0.878
/content/drive/MyDrive/pro/8701/roberta_model/saved_weights_2


([0.8401526717557252, 0.9198473282442748, 0.9658015267175573],
 [0.8827122785583384, 0.8711056811240073, 0.8778252901649358])

In [17]:
def evaluate(model, test_data):

    test = Dataset(test_data)
    pred_label = []
    test_dataloader = torch.utils.data.DataLoader(test, batch_size=8)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():

        for test_input, test_label in test_dataloader:

              test_label = test_label.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask)
              pred = output[0].argmax(dim=1)
              # print(pred)
              # print(test_label)
              pred_label.append(pred)
              acc = (output[0].argmax(dim=1) == test_label).sum().item()
              total_acc_test += acc
    
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')
    return pred_label


#load weights of best model
model2 = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/pro/8701/roberta_model/saved_weights_1')
tokenizer2 = AutoTokenizer.from_pretrained('/content/drive/MyDrive/pro/8701/roberta_model/saved_weights_1')

evaluate(model2, df_test)
# evaluate(model_test, df_train.iloc[:10])

Test Accuracy:  0.866


[]